In [3]:
from Project.Backend import initDB, shutdownDB, displayDB, getDB
from Project.Backend import handle_request
initDB("test.db")
displayDB(getDB(), display)
shutdownDB()

,CommentID,PostID,UserID,Content,ParentID,TargetID,CommentTime


,UserID,PostID,FavoriteTime


,UserID,CommentID,LikeTime


,PostID,UserID,Title,PublishTime


,PostID,TopicID


...

,TargetID,PostID,TargetName,Description


,TopicID,TopicName,CreateTime


,UserID,Nickname,Level,RegisterTime


,LogID,UserID,ActionType,TargetType,TargetID,ActionTime


,name,seq


In [2]:
from Project.Tests import DBTest
testcase = DBTest(display)
testcase.autotest()
testcase.display()

'进行测试: reg_user'

'通过测试: reg_user'

'进行测试: reg_topic'

'通过测试: reg_topic'

'进行测试: post_post'

...

,TargetID,PostID,TargetName,Description
0,1,1,game score,this is a ****ing game which you can give him ...


,TopicID,TopicName,CreateTime
0,1,tp1,2025-04-12 12:51:39
1,2,tp2,2025-04-12 12:51:39
2,3,tp3,2025-04-12 12:51:39
3,4,tpA,2025-04-12 12:51:39
4,5,tpB,2025-04-12 12:51:39
5,6,tpC,2025-04-12 12:51:39


,UserID,Nickname,Level,RegisterTime
0,1,user1,1,2025-04-12 12:51:39
1,2,user2,1,2025-04-12 12:51:39
2,3,user3,1,2025-04-12 12:51:39
3,4,user4,1,2025-04-12 12:51:39
4,5,userA,1,2025-04-12 12:51:39
5,6,userB,1,2025-04-12 12:51:39
6,7,userC,1,2025-04-12 12:51:39
7,8,userD,1,2025-04-12 12:51:39
8,9,userE,1,2025-04-12 12:51:39


,LogID,UserID,ActionType,TargetType,TargetID,ActionTime
0,1,2,CREATE_POST,post,1,2025-04-12 12:51:39
1,2,4,SCORE,target,1,2025-04-12 12:51:39
2,3,4,COMMENT,comment,1,2025-04-12 12:51:39
3,4,5,SCORE,target,1,2025-04-12 12:51:39
4,5,5,COMMENT,comment,2,2025-04-12 12:51:39
5,6,3,COMMENT,comment,3,2025-04-12 12:51:39


,name,seq
0,User,9
1,Topic,6
2,Post,1
3,UserActionLog,6
4,ScoreTarget,1
5,ScoreRecord,2
6,Comment,3
